# Hyperparameter Tuning:

The process of adjusting the _hyperparameters_ of a machine learning model to enhance its performance is known as hyperparameter tuning. Hyperparameters, which are parameters governing the model's training process but not learned from the data, are typically manually configured during the design phase and closely tied to the specific task.

Selection of the optimal hyperparameters is based on the desired system output, with specific metrics utilized to assess its real-time behavior. While metric selection varies depending on the specific context and application, some common metrics employed in classification problems include accuracy, precision, recall, and the F1 score.

Hyperparameter tuning, or optimization, can be categorized as either SOHT (Single Objective Hyperparameters Tuning) or MOHT (Multiple-Objectives Hyperparameters Tuning). In SOHT, the emphasis is on optimizing a single objective, such as model accuracy. Conversely, in multi-objective hyperparameter tuning (MOHT), hyperparameters are optimized to satisfy multiple, often conflicting objectives. MOHT can be accomplished through various approaches, with the option of consolidating multiple objectives into a single one sometimes proving to be a viable strategy.

MOHT is useful in situations where we want to balance different performance metrics, such as accuracy, precision, recall, and training time. In the context of an IDS, we may want to optimize the model to reduce the number of false negatives, compared to the number of false positives that will not impact the security of the network.

Some examples of SOHT can be:
- Reduce the number of false negatives
- Keep prediction time as low as possible

Some examples of MOHT can be:
- Optimizing a machine learning model for both accuracy and fairness.
- Optimizing a machine learning model for both accuracy and speed.
- Optimizing a machine learning model for both accuracy and low CPU usage.
- Optimizing a machine learning model for low false negatives and high recall.

### More on Hyperparameters Tuning

In machine learning models like the Naive Bayes classifier and SVM, hyperparameters are settings that are not learned from the data but are crucial for model performance. Manually tuning these hyperparameters can be time-consuming and might not lead to optimal results. Self-tuning hyperparameters involve mechanisms that automatically find the best hyperparameter settings for your models during the planning phase of the MAPE-K loop. Here's how it can be implemented:

- **Bayesian Optimization**: It's a probabilistic model-based approach that builds a surrogate model of the hyperparameter-performance relationship. It then uses an acquisition function to decide which hyperparameters to try next. Over time, Bayesian optimization adapts to discover the hyperparameters that yield the best performance.

- **Grid Search with Evaluation**: You can set up a grid search where different hyperparameters are systematically tested within predefined ranges. Computationally expensive. In the planning phase, the system can perform a grid search with occasional model evaluation to find the best hyperparameters.

- **Random Search**: Random combinations of hyperparameters are evaluated. Random search can be less intensive than grid search and may lead to good hyperparameter choices.

Hyperparameters tuning and Cross Validation (CV) are strictly correlated.

**Cross-Validation**: Cross-validation is a technique used to assess the performance and generalization ability of a machine learning model. It involves dividing the dataset into multiple subsets (known as _folds_) and iteratively training and testing the model using different combinations of these folds.

**Hyperparameter Tuning within Cross-Validation**: To find the best hyperparameters for a model, it's common to perform hyperparameter tuning within each fold of a cross-validation procedure. In this setup, you search for the best hyperparameters while training on the training portion of the current fold and then evaluate the model's performance on the validation portion of the fold.
This process is repeated for each fold, and the hyperparameters that yield the best average performance across the folds are selected as the final hyperparameters for the model. This approach ensures that the selected hyperparameters generalize well to different data partitions.

# Hyperparameters tuning and self-adaptivity

## Monitor Phase:

Collect data from the IDS's operations on the KDDTest+ dataset first, and then on synthetic network traffic. This data should include information on detection results (e.g., true positives, false negatives), response times, and resource utilization. From the collected data, define performance metrics that are meaningful for the IDS. Common metrics include detection accuracy, false positive rate, false negative rate, and response time. These have already been selected for this specific IDS, but additional ones can be used to identify new trends in the system.
In the Monitoring phase, the IDS continuously collects data about network traffic, system performance and security incidents.
Also gather data on the current threat landscape, including emerging threats, attack patterns, and vulnerabilities.
Monitor the computational resources available.

#### What to monitor

To detect the actual need for self-adaptation, metrics need to be identified to evaluate how _up-to-date_ the system is. These metrics are simple measurements that will be fed to the _analysis_ component that will identify if changes in trends may require adaptations. The following is a non-exhaustive list of what can be monitored in the system:

1. **Performance metrics**: evaluate how the systems perform in terms of the actual intrusion detection.
    - **Detection accuracy**: overall accuracy of the IDS.
    - **Confusion matrix**: a representation of the positives and negatives [TP FN / FP TN]
    - **False (True) positives**: rate at which the IDS produce false (true) alarms.
    - **False (True) negatives**: rate at which che IDS marks traffic as 'normal' correctly (incorrectly).
    - **Response time**: measures the time it takes for the IDS to produce a prediction.
    - **Training time**: how long it takes for the classifier to be trained from data.
    - **Model stability**: This metric is used to ensure the function of the performance tends to be constant in time. A drop of performance is alarming.

2. **Feedback from analysts**: all the metrics above can be updated in complete autonomy in testing, but in real life applications, feedback will come from the classifiers themselves, as well as from analysts that manually classify misclassified connections.

3. **Network data and traffic**: evaluate the properties of the incoming traffic.
    - **Data distribution**: monitor the distribution of network data, including traffic patterns, protocols, attack prevalence. High volumes of data may require higher response times to ensure the IDS does not behave as the bottleneck of the network.
    - **Data quality**: assess the quality of the data collected, inconsistencies or preprocessing problems. This may be useful to identify when the IDS is lacking accuracy or when problems happen before the classification process.
4. **Hardware and software infrastructures**: evaluate the properties of the infrastructure in which the IDS operates.
    - **Resource availability**: keep track of the computational resources, CPU, GPU etc.
5. **Feature engineering**: periodically evaluate the relevance of the features used for detection and adapt the feature set as needed.
6. **Thresholds**: monitor if thresholds have been changed and thus require adaptation.
7. **Train set**: changes in the structure of the train set can be identified. Changes like the _number of features_ and the _number of samples_ which can also be linked to the changes in the features' quality.

### Analyze Phase:

**Analyze component**: this is a fundamental component of the self-adaptive loop in the IDS's MAPE-K architecture. In this phase, the system continuously evaluates the performance of the IDS and collects valuable insights from the data and feedback. The primary tasks of these phases include:

- Compare _baseline_ values (set during design phase) of the metrics with current values to identify possible trends and evaluate if certain thresholds have been surpassed.
- Evaluate if changes in the datasets lead to new optimal features.
- Evaluate if the dataset itself was changed significantly.
- Compare traffic volumes, patterns or other properties of the incoming traffic with historical patterns to evaluate if a drift in traffic distribution is taking place. 
- Evaluate the resource utilization of hardware/software components and identify possible critical situations.
- Evaluate if too many resources are being used compared to baseline values.

**A specific analysis case**: When traffic volumes change or analysts provide feedback the IDS may discover new attacks; if that's the case, evaluate if the expansion of the IDS and subsequent training of a new layer may be useful.

### Plan Phase:

The bulk of the self-adaptation happens here. In this phase, the issues identified in the _analysis_ phase are translated in objectives to reach, then actions to take on the IDS.

1. **Target identification**:
The system is composed of multiple layers, the first thing is to understand if both layers need tuning or if the changes involve only one of the two.
A single adaptation may be needed in cases where a performance drop can be linked to one single layer or time to process increases only in one of the two. However, in real applications the incorrect classification happens only at the end of the pipeline. Another instance in which single layer optimization may be needed is it a single layer uses too many resources or time to process the samples.
In case of dataset expansions, target identifications will be harder since new preprocessing will most likely be needed and subsequent fine-tuning of the whole IDS may be necessary.

2. **Mapping**:
First, a **metric <--> hyperparameter** mapping needs to be done; assigning each metric to one or more hyperparameters allows understanding how an increase/decrease of a numeric hyperparameter, or the change in categorical value, will affect the performance of the classifier. For example, the drastic reduction of support vectors number in an SVM will most likely lead to a decrease in precision. Identify **what** hyperparameter to tweak is the first step in performing precise hyperparameter tuning.

3. **Objective(s) identification**:
Understanding the system state that needs to be reached is the key to achieving good self-adaptation. Performing a **metric change <--> objective** mapping will be required to understand the starting and ending point of the adaptation process. First we need to evaluate what direction we want the system to head to, and then code the direction as a finite set of objectives to feed the tuning function(s).

4. **Hyperparameter tuning**:
Once the objectives have been identified, SOHT or MOHT is performed as well as cross validation to evaluate the goodness of these new hyperparameters in our system. The output of this process is a set of hyperparameters.

### Execute Phase:
**Model Reconfiguration**: the IDS should reconfigure its models based on the chosen hyperparameters. It may involve retraining the models using the KDDTest+ dataset with the optimized set of hyperparameters.
After the models have been reconfigured, the Monitor phase should continue to detect the evolution of the metrics and enable new iterations of the MAPE-K loop.

### Automation:
Automate the self-adaptation process. The IDS should be able to perform most of these steps automatically without requiring manual intervention. However, manual intervention in IDS is unavoidable, human supervision can be reduced but not eliminated:

1. Operators annotate intrusions when the system encounters difficulties in classifying specific cases or when it mistakenly labels anomalies. This annotation process helps in creating **new data elements** that can be incorporated into the dataset.
2. Operators can **manually trigger** the self-adaptive loop.
3. Adaptation rules such as **thresholds for accuracy** or **confusion matrix** values can be manually tweaked.
4. _Baseline values_ for the thresholds are manually modified, the triggers for self-adaptation change after manual intervention.

## When to self-adapt

Here's a non-exhaustive list of scenarios in which the self-adaptation loop will be triggered:

1. **Initial model training**: when the IDS is built, the best settings need to be found according to the data present.
2. **Adapting to new threats**: as new attacks are documented and added to the knowledge base, the quality of the samples, their representation, or their volume may change. In this context new parameters may be necessary.
3. **Changing network conditions**: fluctuations in the volume of data can impact the IDS's performance, traffic intensifies or slows down. Hyperparameter tuning may help the system adapt to new conditions.
4. **Data drift**: Over time, the concept or distribution of normal and malicious network traffic may shift. This drift may be gradual but still requires adapting via tuning.
5. **Model updates**: When (and if) the models deployed in the IDS change, new hyperparameters tuning is necessary. A new model choice requires new considerations.
6. **Performance degradation**: general decrease of performance due to changes of threats, data patterns.
7. **Feature engineering**: new features are introduced/modified, new hyperparameters may perform better under these conditions.



In [273]:
import numpy as np 
import pandas as pd
import copy
import pickle
import joblib

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler as under_sam

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import matthews_corrcoef, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

### ICFS function
Takes a dataframe as parameter and saves to file all the features necessary to describe DoS+Probe and U2R+R2L

In [274]:
def pearson_correlated_features(x, y, threshold):
    y['target'] = y['target'].astype(int)

    for p in x.columns:
        x[p] = x[p].astype(float)

    # Ensure y is a DataFrame for consistency
    if isinstance(y, pd.Series):
        y = pd.DataFrame(y, columns=['target'])

    # Calculate the Pearson's correlation coefficients between features and the target variable(s)
    corr_matrix = x.corrwith(y['target'])

    # Select features with correlations above the threshold
    selected_features = x.columns[corr_matrix.abs() > threshold].tolist()

    return selected_features

In [275]:
def compute_set_difference(df1, df2):
    # Create a new DataFrame containing the set difference of the two DataFrames.
    df_diff = df1[~df1.index.isin(df2.index)]
    # Return the DataFrame.
    return df_diff

In [276]:
def perform_icfs(x_train):
    # now ICFS only on the numerical features
    num_train = copy.deepcopy(x_train)
    del num_train['protocol_type']
    del num_train['service']
    del num_train['flag']

    target = pd.DataFrame()
    target['target'] = np.array([1 if x != 'normal' else 0 for x in num_train['label']])
    num_train = pd.concat([num_train, target], axis=1)

    # These are how attacks are categorized in the trainset
    dos_list = ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop']
    probe_list = ['ipsweep', 'portsweep', 'satan', 'nmap']
    u2r_list = ['loadmodule', 'perl', 'rootkit', 'buffer_overflow']
    r2l_list = ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'phf', 'spy', 'warezclient', 'warezmaster']
    normal = ['normal']

    # useful sub-sets
    x_normal = num_train[num_train['label'].isin(normal)]
    x_u2r = num_train[num_train['label'].isin(u2r_list)]
    x_r2l = num_train[num_train['label'].isin(r2l_list)]
    x_dos = num_train[num_train['label'].isin(dos_list)]
    x_probe = num_train[num_train['label'].isin(probe_list)]

    # start the ICFS with l1

    # features for dos
    dos = copy.deepcopy(num_train)
    del dos['target']
    y = np.array([1 if x in dos_list else 0 for x in dos['label']])
    y_dos = pd.DataFrame(y, columns=['target'])
    del dos['label']
    dos_all = pearson_correlated_features(dos, y_dos, 0.1)
    print(dos_all)

    # features for probe
    probe = copy.deepcopy(num_train)
    del probe['target']
    y = np.array([1 if x in probe_list else 0 for x in probe['label']])
    y_probe = pd.DataFrame(y, columns=['target'])
    del probe['label']
    probe_all = pearson_correlated_features(probe, y_probe, 0.1)
    print(probe_all)

    # intersect for the optimal features
    set_dos = set(dos_all)
    set_probe = set(probe_all)

    comm_features_l1 = set_probe & set_dos

    print('common features to train l1: ', comm_features_l1)

    # now l2 needs the features to describe the difference between rare attacks and normal traffic

    # features for u2r
    u2r = pd.concat([x_u2r, x_normal], axis=0)
    del u2r['target']
    y = np.array([1 if x in u2r_list else 0 for x in u2r['label']])
    y_u2r = pd.DataFrame(y, columns=['target'])
    del u2r['label']
    u2r_all = pearson_correlated_features(u2r, y_u2r, 0.01)
    print(u2r_all)

    # features for r2l
    r2l = pd.concat([x_r2l, x_normal], axis=0)
    del r2l['target']
    y = np.array([1 if x in r2l_list else 0 for x in r2l['label']])
    y_r2l = pd.DataFrame(y, columns=['target'])
    del r2l['label']
    r2l_all = pearson_correlated_features(r2l, y_r2l, 0.01)
    print(r2l_all)

    # intersect for the optimal features
    set_r2l = set(r2l_all)
    set_u2r = set(u2r_all)

    comm_features_l2 = set_r2l & set_u2r
    # print('Common features to train l2: ', len(common_features_l2), common_features_l2)

    with open('NSL-KDD Files/NSL_features_l1.txt', 'w') as g:
        for a, x in enumerate(comm_features_l1):
            if a < len(comm_features_l1) - 1:
                g.write(x + ',' + '\n')
            else:
                g.write(x)

    # read the common features from file
    with open('NSL-KDD Files/NSL_features_l2.txt', 'w') as g:
        for a, x in enumerate(comm_features_l2):
            if a < len(comm_features_l2) - 1:
                g.write(x + ',' + '\n')
            else:
                g.write(x)

# Main implementation

In [277]:
# loading the train set
df_train = pd.read_csv('NSL-KDD Original Datasets/KDDTrain+.txt', sep=",", header=None)
df_train = df_train[df_train.columns[:-1]]  # tags column
titles = pd.read_csv('NSL-KDD Original Datasets/Field Names.csv', header=None)
label = pd.Series(['label'], index=[41])
titles = pd.concat([titles[0], label])
df_train.columns = titles.to_list()
df_train = df_train.drop(['num_outbound_cmds'],axis=1)
df_train_original = df_train
# df_train_original

In [278]:
# load test set
df_test = pd.read_csv('NSL-KDD Original Datasets/KDDTest+.txt', sep=",", header=None)
df_test = df_test[df_test.columns[:-1]]
df_test.columns = titles.to_list()
df_test = df_test.drop(['num_outbound_cmds'],axis=1)
df_test_original = df_test
# df_test_original

### Execution Parameters

In [279]:
EXPORT_MODELS = 1
EXPORT_DATASETS = 1
EXPORT_PCA = 1
EXPORT_ENCODERS = 1

### Perform ICFS if needed

In [280]:
# It is possible to compute the ICFS again

# perform_icfs(df_train_original)

# DoS + Probe classifier (NBC)

In [281]:
# list of single attacks 
dos_attacks = ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop', 'worm', 'apache2', 'mailbomb', 'processtable', 'udpstorm']
probe_attacks = ['ipsweep', 'mscan', 'nmap', 'portsweep', 'saint', 'satan']
r2l_attacks = ['guess_passwd', 'ftp_write', 'imap', 'phf', 'multihop', 'warezmaster',
                'snmpguess', 'spy', 'warezclient', 'httptunnel', 'named', 'sendmail', 'snmpgetattack', 'xlock', 'xsnoop']
u2r_attacks = ['buffer_overflow', 'loadmodule', 'perl', 'ps', 'rootkit', 'sqlattack', 'xterm'] 

# list of attack classes split according to detection layer
dos_probe_list = ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop', 'ipsweep', 'nmap', 'portsweep', 'satan']
dos_probe_test = ['apache2', 'mailbomb', 'processtable', 'udpstorm', 'mscan', 'saint']
u2r_r2l_list = ['guess_passwd', 'ftp_write', 'imap', 'phf', 'multihop', 'warezmaster',
                'snmpguess', 'spy', 'warezclient', 'buffer_overflow', 'loadmodule', 'rootkit', 'perl']
u2r_r2l_test = ['httptunnel', 'named', 'sendmail', 'snmpgetattack', 'xlock', 'xsnoop', 'ps', 'xterm', 'sqlattack']
normal_list = ['normal']
categorical_features = ['protocol_type', 'service', 'flag']

# load the features obtained with ICFS for both layer 1 and layer 2
with open('NSL-KDD Files/NSL_features_l1.txt', 'r') as f:
    common_features_l1 = f.read().split(',')

with open('NSL-KDD Files/NSL_features_l2.txt', 'r') as f:
    common_features_l2 = f.read().split(',')
    
df_train_and_validate = copy.deepcopy(df_train_original)
df_test = copy.deepcopy(df_test_original)

In [282]:
# split in test and validation set for BOTH layers
df_train_original, df_val_original = train_test_split(df_train_and_validate, test_size=0.2, random_state=42)

# dataframes specifically for layer 1
df_train = copy.deepcopy(df_train_original)
df_val = copy.deepcopy(df_val_original)

# set the target variables accordingly
y_train = np.array([1 if x in (dos_attacks+probe_attacks) else 0 for x in df_train['label']])
y_val = np.array([1 if x in (dos_attacks+probe_attacks) else 0 for x in df_val['label']])

# this dataframe contains the whole train set 
df_train = df_train.drop(['label'],axis=1)
df_train = df_train.reset_index().drop(['index'], axis=1)
df_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,http,SF,214,14939,0,0,0,0,...,52,255,1.00,0.00,0.02,0.06,0.0,0.0,0.0,0.0
1,0,tcp,private,S0,0,0,0,0,0,0,...,255,2,0.01,0.06,0.00,0.00,1.0,1.0,0.0,0.0
2,0,tcp,http,REJ,0,0,0,0,0,0,...,255,8,0.03,0.06,0.00,0.00,0.0,0.0,1.0,1.0
3,0,tcp,http,SF,257,259,0,0,0,0,...,255,255,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
4,0,udp,other,SF,516,4,0,0,0,0,...,255,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100773,0,tcp,echo,RSTO,0,0,0,0,0,0,...,255,4,0.02,0.09,0.00,0.00,0.0,0.0,1.0,1.0
100774,0,tcp,telnet,S0,0,0,0,0,0,0,...,255,4,0.02,0.07,0.01,0.00,1.0,1.0,0.0,0.0
100775,0,tcp,http,REJ,0,0,0,0,0,0,...,255,6,0.02,0.07,0.00,0.00,0.0,0.0,1.0,1.0
100776,0,tcp,http,SF,309,4281,0,0,0,0,...,21,255,1.00,0.00,0.05,0.05,0.0,0.0,0.0,0.0


In [283]:
# this dataframe contains the whole validation set
df_val = df_val.drop(['label'],axis=1)
df_val = df_val.reset_index().drop(['index'], axis=1)

df_val

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,udp,domain_u,SF,36,0,0,0,0,0,...,67,171,1.00,0.00,1.00,0.01,0.00,0.00,0.00,0.0
1,0,tcp,http,S0,0,0,0,0,0,0,...,255,44,0.17,0.05,0.01,0.00,1.00,1.00,0.00,0.0
2,0,tcp,pop_3,S0,0,0,0,0,0,0,...,255,20,0.08,0.06,0.00,0.00,1.00,1.00,0.00,0.0
3,0,tcp,private,REJ,0,0,0,0,0,0,...,255,27,0.11,0.07,0.00,0.00,0.00,0.00,1.00,1.0
4,0,tcp,private,RSTR,0,0,0,0,0,0,...,134,1,0.01,0.64,0.64,0.00,0.04,0.00,0.63,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25190,0,tcp,http,SF,199,944,0,0,0,0,...,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
25191,0,icmp,eco_i,SF,8,0,0,0,0,0,...,2,129,1.00,0.00,1.00,0.50,0.00,0.00,0.00,0.0
25192,0,tcp,ftp_data,SF,12983,0,0,0,0,0,...,140,9,0.06,0.04,0.06,0.00,0.00,0.00,0.02,0.0
25193,2,tcp,smtp,SF,813,329,0,0,0,0,...,255,141,0.55,0.06,0.00,0.00,0.02,0.02,0.00,0.0


In [284]:
# now the real processing for layer 1 starts
X_train = df_train[common_features_l1]
X_train

,logged_in,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_srv_rerror_rate,dst_host_rerror_rate
0,1,16,0.0,0.0,0.0,1.00,0.00,0.17,255,1.00,0.00,0.02,0.06,0.0,0.0,0.0,0.0
1,0,142,1.0,0.0,0.0,0.01,0.06,0.00,2,0.01,0.06,0.00,0.00,1.0,1.0,0.0,0.0
2,0,273,0.0,1.0,1.0,0.03,0.06,0.00,8,0.03,0.06,0.00,0.00,0.0,0.0,1.0,1.0
3,1,20,0.0,0.0,0.0,1.00,0.00,0.00,255,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
4,0,274,0.0,0.0,0.0,1.00,0.00,0.00,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100773,0,258,0.0,1.0,1.0,0.02,0.07,0.00,4,0.02,0.09,0.00,0.00,0.0,0.0,1.0,1.0
100774,0,24,1.0,0.0,0.0,0.17,0.08,0.00,4,0.02,0.07,0.01,0.00,1.0,1.0,0.0,0.0
100775,0,258,0.0,1.0,1.0,0.02,0.07,0.00,6,0.02,0.07,0.00,0.00,0.0,0.0,1.0,1.0
100776,1,5,0.0,0.0,0.0,1.00,0.00,0.00,255,1.00,0.00,0.05,0.05,0.0,0.0,0.0,0.0


In [285]:
X_validate = df_val[common_features_l1]
X_validate

,logged_in,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_srv_rerror_rate,dst_host_rerror_rate
0,0,2,0.0,0.0,0.0,1.00,0.00,0.75,171,1.00,0.00,1.00,0.01,0.00,0.00,0.0,0.00
1,0,42,1.0,0.0,0.0,0.26,0.10,0.00,44,0.17,0.05,0.01,0.00,1.00,1.00,0.0,0.00
2,0,284,1.0,0.0,0.0,0.07,0.06,0.00,20,0.08,0.06,0.00,0.00,1.00,1.00,0.0,0.00
3,0,110,0.0,1.0,1.0,0.07,0.06,0.00,27,0.11,0.07,0.00,0.00,0.00,0.00,1.0,1.00
4,0,1,0.0,1.0,1.0,1.00,0.00,0.00,1,0.01,0.64,0.64,0.00,0.04,0.00,1.0,0.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25190,1,11,0.0,0.0,0.0,1.00,0.00,0.15,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
25191,0,1,0.0,0.0,0.0,1.00,0.00,1.00,129,1.00,0.00,1.00,0.50,0.00,0.00,0.0,0.00
25192,0,2,0.0,0.0,0.0,1.00,0.00,0.00,9,0.06,0.04,0.06,0.00,0.00,0.00,0.0,0.02
25193,1,1,0.0,0.0,0.0,1.00,0.00,1.00,141,0.55,0.06,0.00,0.00,0.02,0.02,0.0,0.00


In [286]:
# 2 one-hot encoders, one for the features of layer1 and one for the features of layer2
ohe = OneHotEncoder(handle_unknown='ignore')
ohe2 = OneHotEncoder(handle_unknown='ignore')
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()

In [287]:
# scaling the train set for layer1
df_minmax = scaler1.fit_transform(X_train)
X_train = pd.DataFrame(df_minmax, columns=X_train.columns)

X_train

,logged_in,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_srv_rerror_rate,dst_host_rerror_rate
0,1.0,0.031311,0.0,0.0,0.0,1.00,0.00,0.17,1.000000,1.00,0.00,0.02,0.06,0.0,0.0,0.0,0.0
1,0.0,0.277886,1.0,0.0,0.0,0.01,0.06,0.00,0.007843,0.01,0.06,0.00,0.00,1.0,1.0,0.0,0.0
2,0.0,0.534247,0.0,1.0,1.0,0.03,0.06,0.00,0.031373,0.03,0.06,0.00,0.00,0.0,0.0,1.0,1.0
3,1.0,0.039139,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
4,0.0,0.536204,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100773,0.0,0.504892,0.0,1.0,1.0,0.02,0.07,0.00,0.015686,0.02,0.09,0.00,0.00,0.0,0.0,1.0,1.0
100774,0.0,0.046967,1.0,0.0,0.0,0.17,0.08,0.00,0.015686,0.02,0.07,0.01,0.00,1.0,1.0,0.0,0.0
100775,0.0,0.504892,0.0,1.0,1.0,0.02,0.07,0.00,0.023529,0.02,0.07,0.00,0.00,0.0,0.0,1.0,1.0
100776,1.0,0.009785,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,1.00,0.00,0.05,0.05,0.0,0.0,0.0,0.0


In [288]:
# scaling the validation set for layer1
df_minmax_val = scaler1.transform(X_validate)
X_validate = pd.DataFrame(df_minmax_val, columns=X_validate.columns)

X_validate

,logged_in,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_srv_rerror_rate,dst_host_rerror_rate
0,0.0,0.003914,0.0,0.0,0.0,1.00,0.00,0.75,0.670588,1.00,0.00,1.00,0.01,0.00,0.00,0.0,0.00
1,0.0,0.082192,1.0,0.0,0.0,0.26,0.10,0.00,0.172549,0.17,0.05,0.01,0.00,1.00,1.00,0.0,0.00
2,0.0,0.555773,1.0,0.0,0.0,0.07,0.06,0.00,0.078431,0.08,0.06,0.00,0.00,1.00,1.00,0.0,0.00
3,0.0,0.215264,0.0,1.0,1.0,0.07,0.06,0.00,0.105882,0.11,0.07,0.00,0.00,0.00,0.00,1.0,1.00
4,0.0,0.001957,0.0,1.0,1.0,1.00,0.00,0.00,0.003922,0.01,0.64,0.64,0.00,0.04,0.00,1.0,0.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25190,1.0,0.021526,0.0,0.0,0.0,1.00,0.00,0.15,1.000000,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
25191,0.0,0.001957,0.0,0.0,0.0,1.00,0.00,1.00,0.505882,1.00,0.00,1.00,0.50,0.00,0.00,0.0,0.00
25192,0.0,0.003914,0.0,0.0,0.0,1.00,0.00,0.00,0.035294,0.06,0.04,0.06,0.00,0.00,0.00,0.0,0.02
25193,1.0,0.001957,0.0,0.0,0.0,1.00,0.00,1.00,0.552941,0.55,0.06,0.00,0.00,0.02,0.02,0.0,0.00


In [289]:
# perform One-hot encoding for the train set
label_enc = ohe.fit_transform(df_train.iloc[:,1:4])
label_enc.toarray()
new_labels = ohe.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_train = pd.concat([X_train, df_enc], axis=1)

X_train

,logged_in,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,1.0,0.031311,0.0,0.0,0.0,1.00,0.00,0.17,1.000000,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.277886,1.0,0.0,0.0,0.01,0.06,0.00,0.007843,0.01,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.534247,0.0,1.0,1.0,0.03,0.06,0.00,0.031373,0.03,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.039139,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.536204,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100773,0.0,0.504892,0.0,1.0,1.0,0.02,0.07,0.00,0.015686,0.02,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100774,0.0,0.046967,1.0,0.0,0.0,0.17,0.08,0.00,0.015686,0.02,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
100775,0.0,0.504892,0.0,1.0,1.0,0.02,0.07,0.00,0.023529,0.02,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100776,1.0,0.009785,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [290]:
# perform One-hot encoding for the validation set
label_enc = ohe.transform(df_val.iloc[:,1:4])
label_enc.toarray()
new_labels = ohe.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_validate = pd.concat([X_validate, df_enc], axis=1)

X_validate

,logged_in,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,0.003914,0.0,0.0,0.0,1.00,0.00,0.75,0.670588,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.082192,1.0,0.0,0.0,0.26,0.10,0.00,0.172549,0.17,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.555773,1.0,0.0,0.0,0.07,0.06,0.00,0.078431,0.08,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.215264,0.0,1.0,1.0,0.07,0.06,0.00,0.105882,0.11,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.001957,0.0,1.0,1.0,1.00,0.00,0.00,0.003922,0.01,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25190,1.0,0.021526,0.0,0.0,0.0,1.00,0.00,0.15,1.000000,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
25191,0.0,0.001957,0.0,0.0,0.0,1.00,0.00,1.00,0.505882,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
25192,0.0,0.003914,0.0,0.0,0.0,1.00,0.00,0.00,0.035294,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
25193,1.0,0.001957,0.0,0.0,0.0,1.00,0.00,1.00,0.552941,0.55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [291]:
# do the same for testset
y_test = np.array([1 if x in (dos_attacks+probe_attacks) else 0 for x in df_test['label']])

df_test = df_test.drop(['label'],axis=1)
df_test = df_test.reset_index().drop(['index'], axis=1)
df_test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,private,REJ,0,0,0,0,0,0,...,255,10,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00
1,0,tcp,private,REJ,0,0,0,0,0,0,...,255,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,134,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,3,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,29,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,100,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00
22540,0,tcp,http,SF,317,938,0,0,0,0,...,197,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,255,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00


In [292]:
X_test = df_test[common_features_l1]

X_test

,logged_in,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_srv_rerror_rate,dst_host_rerror_rate
0,0,229,0.0,1.0,1.0,0.04,0.06,0.00,10,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00
1,0,136,0.0,1.0,1.0,0.01,0.06,0.00,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00
2,0,1,0.0,0.0,0.0,1.00,0.00,0.00,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00
3,0,1,0.0,0.0,0.0,1.00,0.00,1.00,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00
4,0,1,0.0,1.0,0.5,1.00,0.00,0.75,86,0.31,0.17,0.03,0.02,0.00,0.0,0.71,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,1,1,0.0,0.0,0.0,1.00,0.00,0.00,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00
22540,1,2,0.0,0.0,0.0,1.00,0.00,0.18,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00
22541,1,5,0.0,0.0,0.0,1.00,0.00,0.20,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07
22542,0,4,0.0,0.0,0.0,1.00,0.00,0.33,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00


In [293]:
df_minmax = scaler1.transform(X_test)
X_test = pd.DataFrame(df_minmax, columns=X_test.columns)
X_test

,logged_in,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_srv_rerror_rate,dst_host_rerror_rate
0,0.0,0.448141,0.0,1.0,1.0,0.04,0.06,0.00,0.039216,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00
1,0.0,0.266145,0.0,1.0,1.0,0.01,0.06,0.00,0.003922,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00
2,0.0,0.001957,0.0,0.0,0.0,1.00,0.00,0.00,0.337255,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00
3,0.0,0.001957,0.0,0.0,0.0,1.00,0.00,1.00,0.223529,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00
4,0.0,0.001957,0.0,1.0,0.5,1.00,0.00,0.75,0.337255,0.31,0.17,0.03,0.02,0.00,0.0,0.71,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,1.0,0.001957,0.0,0.0,0.0,1.00,0.00,0.00,0.552941,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00
22540,1.0,0.003914,0.0,0.0,0.0,1.00,0.00,0.18,1.000000,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00
22541,1.0,0.009785,0.0,0.0,0.0,1.00,0.00,0.20,1.000000,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07
22542,0.0,0.007828,0.0,0.0,0.0,1.00,0.00,0.33,0.988235,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00


In [294]:
label_enc = ohe.transform(df_test.iloc[:,1:4])
label_enc.toarray()
new_labels = ohe.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_test = pd.concat([X_test, df_enc], axis=1)
X_test

,logged_in,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,0.448141,0.0,1.0,1.0,0.04,0.06,0.00,0.039216,0.04,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.266145,0.0,1.0,1.0,0.01,0.06,0.00,0.003922,0.00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.001957,0.0,0.0,0.0,1.00,0.00,0.00,0.337255,0.61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.001957,0.0,0.0,0.0,1.00,0.00,1.00,0.223529,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.001957,0.0,1.0,0.5,1.00,0.00,0.75,0.337255,0.31,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,1.0,0.001957,0.0,0.0,0.0,1.00,0.00,0.00,0.552941,0.72,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22540,1.0,0.003914,0.0,0.0,0.0,1.00,0.00,0.18,1.000000,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22541,1.0,0.009785,0.0,0.0,0.0,1.00,0.00,0.20,1.000000,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22542,0.0,0.007828,0.0,0.0,0.0,1.00,0.00,0.33,0.988235,0.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [295]:
print('Shape of the whole train set: ', X_train.shape)
print('Shape of its targets: ', y_train.shape)
print('Shape of the whole test set: ', X_test.shape)
print('Shape of its targets: ', y_test.shape)

Shape of the whole train set:  (100778, 101)
Shape of its targets:  (100778,)
Shape of the whole test set:  (22544, 101)
Shape of its targets:  (22544,)


In [296]:
# Export the dataset for training layer 1
if EXPORT_DATASETS:
    X_train.to_csv('NSL-KDD Encoded Datasets/KDDTrain+_l1.txt', index=False)
    X_validate.to_csv('NSL-KDD Encoded Datasets/KDDValidate+_l1.txt', index=False)
    np.save('NSL-KDD Encoded Datasets/KDDTrain+_l1_targets', y_train)
    np.save('NSL-KDD Encoded Datasets/KDDValidate+_l1_targets', y_val)

### Principal Component Analysis

In [297]:
pca_dos_probe = PCA(n_components=0.95)
X_train_dos_probe = pca_dos_probe.fit_transform(X_train)
X_test_dos_probe = pca_dos_probe.transform(X_test)
X_validate_dos_probe = pca_dos_probe.transform(X_validate)

In [298]:
if EXPORT_PCA:
    # save the pca transformed as well as the transformer
    joblib.dump(X_test_dos_probe, 'NSL-KDD Encoded Datasets/pca_transformed/pca_test1.pkl')
    joblib.dump(X_train_dos_probe, 'NSL-KDD Encoded Datasets/pca_transformed/pca_train1.pkl')
    joblib.dump(X_validate_dos_probe, 'NSL-KDD Encoded Datasets/pca_transformed/pca_validate1.pkl')
    joblib.dump(pca_dos_probe, 'NSL-KDD Encoded Datasets/pca_transformed/layer1_transformer.pkl')

### Building the classifier for the layer1

In [299]:
# Using Random Forest Classifier
# dos_probe_classifier = RandomForestClassifier(n_estimators=100, criterion='gini')

# Using the Naive Bayes Classifier
dos_probe_classifier = GaussianNB()
dos_probe_classifier.fit(X_train_dos_probe, y_train)
predicted = dos_probe_classifier.predict(X_test_dos_probe)

In [300]:
print('Metrics for layer 1:')
print('Confusion matrix: [TP FN / FP TN]\n', confusion_matrix(y_test,predicted))
print('Accuracy = ', accuracy_score(y_test,predicted))
print('F1 Score = ', f1_score(y_test,predicted))
print('Precision = ', precision_score(y_test,predicted))
print('Recall = ', recall_score(y_test,predicted))
print('Shape of the train set for l1: ', X_train_dos_probe.shape)

Metrics for layer 1:
Confusion matrix: [TP FN / FP TN]
 [[9557 3106]
 [ 903 8978]]
Accuracy =  0.8221699787083038
F1 Score =  0.8174823582972911
Precision =  0.7429659053293611
Recall =  0.9086124886145127
Shape of the train set for l1:  (100778, 28)


# R2L+U2R classifier

In [301]:
df_train = copy.deepcopy(df_train_original)
df_test = copy.deepcopy(df_test_original)
df_val = copy.deepcopy(df_val_original)

# load targeted attacks (Normal + r2l + u2r)
df_train = df_train[df_train['label'].isin(normal_list+u2r_attacks+r2l_attacks)]
df_val = df_val[df_val['label'].isin(normal_list+u2r_attacks+r2l_attacks)]

# set the target variables accordingly
y_train = np.array([1 if x in (u2r_attacks+r2l_attacks) else 0 for x in df_train['label']])
y_val = np.array([1 if x in (u2r_attacks+r2l_attacks) else 0 for x in df_val['label']])

df_train = df_train.drop(['label'],axis=1)
df_train = df_train.reset_index().drop(['index'], axis=1)
df_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,http,SF,214,14939,0,0,0,0,...,52,255,1.00,0.00,0.02,0.06,0.0,0.0,0.0,0.0
1,0,tcp,http,SF,257,259,0,0,0,0,...,255,255,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
2,0,udp,other,SF,516,4,0,0,0,0,...,255,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0
3,0,tcp,ftp_data,SF,7940,0,0,0,0,0,...,79,129,0.70,0.05,0.70,0.02,0.0,0.0,0.0,0.0
4,6496,udp,other,SF,147,105,0,0,0,0,...,255,1,0.00,0.62,0.90,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54728,5,tcp,smtp,SF,908,364,0,0,0,0,...,199,148,0.74,0.03,0.01,0.00,0.0,0.0,0.0,0.0
54729,5,tcp,shell,SF,67,1,0,0,0,0,...,255,1,0.00,0.02,0.00,0.00,0.0,0.0,0.0,0.0
54730,0,tcp,http,SF,304,2698,0,0,0,0,...,218,208,0.95,0.01,0.00,0.00,0.0,0.0,0.0,0.0
54731,0,tcp,http,SF,309,4281,0,0,0,0,...,21,255,1.00,0.00,0.05,0.05,0.0,0.0,0.0,0.0


In [302]:
df_val = df_val.drop(['label'],axis=1)
df_val = df_val.reset_index().drop(['index'], axis=1)
df_val

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,udp,domain_u,SF,36,0,0,0,0,0,...,67,171,1.00,0.00,1.00,0.01,0.00,0.00,0.00,0.0
1,0,tcp,http,SF,325,4673,0,0,0,0,...,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
2,0,tcp,smtp,SF,743,335,0,0,0,0,...,138,85,0.62,0.05,0.01,0.00,0.00,0.00,0.00,0.0
3,0,tcp,http,SF,209,29055,0,0,0,0,...,168,255,1.00,0.00,0.01,0.02,0.00,0.00,0.00,0.0
4,0,tcp,ftp_data,SF,567,0,0,0,0,0,...,113,39,0.35,0.04,0.35,0.00,0.02,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13652,0,udp,domain_u,SF,43,70,0,0,0,0,...,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.0
13653,0,udp,domain_u,SF,45,134,0,0,0,0,...,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.0
13654,0,tcp,http,SF,199,944,0,0,0,0,...,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
13655,0,tcp,ftp_data,SF,12983,0,0,0,0,0,...,140,9,0.06,0.04,0.06,0.00,0.00,0.00,0.02,0.0


In [303]:
X_train = df_train[common_features_l2]
X_train

,srv_count,urgent,root_shell,hot,dst_host_srv_diff_host_rate,dst_host_count,logged_in,dst_host_same_src_port_rate,srv_diff_host_rate,num_shells,dst_host_srv_count
0,18,0,0,0,0.06,52,1,0.02,0.17,0,255
1,20,0,0,0,0.00,255,1,0.00,0.00,0,255
2,274,0,0,0,0.00,255,0,1.00,0.00,0,255
3,5,0,0,0,0.02,79,0,0.70,0.00,0,129
4,1,0,0,0,0.00,255,0,0.90,0.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...
54728,4,0,0,0,0.00,199,1,0.01,0.75,0,148
54729,1,0,0,0,0.00,255,0,0.00,0.00,0,1
54730,4,0,0,0,0.00,218,1,0.00,0.00,0,208
54731,5,0,0,0,0.05,21,1,0.05,0.00,0,255


In [304]:
X_validate = df_val[common_features_l2]
X_validate

,srv_count,urgent,root_shell,hot,dst_host_srv_diff_host_rate,dst_host_count,logged_in,dst_host_same_src_port_rate,srv_diff_host_rate,num_shells,dst_host_srv_count
0,4,0,0,0,0.01,67,0,1.00,0.75,0,171
1,20,0,0,0,0.00,255,1,0.00,0.00,0,255
2,2,0,0,0,0.00,138,1,0.01,1.00,0,85
3,3,0,0,0,0.02,168,1,0.01,0.00,0,255
4,1,0,0,0,0.00,113,1,0.35,0.00,0,39
...,...,...,...,...,...,...,...,...,...,...,...
13652,274,0,0,0,0.00,255,0,0.00,0.00,0,254
13653,154,0,0,0,0.00,255,0,0.00,0.01,0,254
13654,13,0,0,0,0.00,255,1,0.00,0.15,0,255
13655,2,0,0,0,0.00,140,0,0.06,0.00,0,9


In [305]:
df_minmax = scaler2.fit_transform(X_train)
X_train = pd.DataFrame(df_minmax, columns=X_train.columns)
X_train

,srv_count,urgent,root_shell,hot,dst_host_srv_diff_host_rate,dst_host_count,logged_in,dst_host_same_src_port_rate,srv_diff_host_rate,num_shells,dst_host_srv_count
0,0.035225,0.0,0.0,0.0,0.06,0.203922,1.0,0.02,0.17,0.0,1.000000
1,0.039139,0.0,0.0,0.0,0.00,1.000000,1.0,0.00,0.00,0.0,1.000000
2,0.536204,0.0,0.0,0.0,0.00,1.000000,0.0,1.00,0.00,0.0,1.000000
3,0.009785,0.0,0.0,0.0,0.02,0.309804,0.0,0.70,0.00,0.0,0.505882
4,0.001957,0.0,0.0,0.0,0.00,1.000000,0.0,0.90,0.00,0.0,0.003922
...,...,...,...,...,...,...,...,...,...,...,...
54728,0.007828,0.0,0.0,0.0,0.00,0.780392,1.0,0.01,0.75,0.0,0.580392
54729,0.001957,0.0,0.0,0.0,0.00,1.000000,0.0,0.00,0.00,0.0,0.003922
54730,0.007828,0.0,0.0,0.0,0.00,0.854902,1.0,0.00,0.00,0.0,0.815686
54731,0.009785,0.0,0.0,0.0,0.05,0.082353,1.0,0.05,0.00,0.0,1.000000


In [306]:
df_minmax = scaler2.transform(X_validate)
X_validate = pd.DataFrame(df_minmax, columns=X_validate.columns)
X_validate

,srv_count,urgent,root_shell,hot,dst_host_srv_diff_host_rate,dst_host_count,logged_in,dst_host_same_src_port_rate,srv_diff_host_rate,num_shells,dst_host_srv_count
0,0.007828,0.0,0.0,0.0,0.01,0.262745,0.0,1.00,0.75,0.0,0.670588
1,0.039139,0.0,0.0,0.0,0.00,1.000000,1.0,0.00,0.00,0.0,1.000000
2,0.003914,0.0,0.0,0.0,0.00,0.541176,1.0,0.01,1.00,0.0,0.333333
3,0.005871,0.0,0.0,0.0,0.02,0.658824,1.0,0.01,0.00,0.0,1.000000
4,0.001957,0.0,0.0,0.0,0.00,0.443137,1.0,0.35,0.00,0.0,0.152941
...,...,...,...,...,...,...,...,...,...,...,...
13652,0.536204,0.0,0.0,0.0,0.00,1.000000,0.0,0.00,0.00,0.0,0.996078
13653,0.301370,0.0,0.0,0.0,0.00,1.000000,0.0,0.00,0.01,0.0,0.996078
13654,0.025440,0.0,0.0,0.0,0.00,1.000000,1.0,0.00,0.15,0.0,1.000000
13655,0.003914,0.0,0.0,0.0,0.00,0.549020,0.0,0.06,0.00,0.0,0.035294


In [307]:
# perform One-hot encoding for the train set
label_enc = ohe2.fit_transform(df_train.iloc[:,1:4])
label_enc.toarray()
new_labels = ohe2.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_train = pd.concat([X_train, df_enc], axis=1)
X_train

,srv_count,urgent,root_shell,hot,dst_host_srv_diff_host_rate,dst_host_count,logged_in,dst_host_same_src_port_rate,srv_diff_host_rate,num_shells,...,flag_OTH,flag_REJ,flag_RSTO,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.035225,0.0,0.0,0.0,0.06,0.203922,1.0,0.02,0.17,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.039139,0.0,0.0,0.0,0.00,1.000000,1.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.536204,0.0,0.0,0.0,0.00,1.000000,0.0,1.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.009785,0.0,0.0,0.0,0.02,0.309804,0.0,0.70,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.001957,0.0,0.0,0.0,0.00,1.000000,0.0,0.90,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54728,0.007828,0.0,0.0,0.0,0.00,0.780392,1.0,0.01,0.75,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
54729,0.001957,0.0,0.0,0.0,0.00,1.000000,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
54730,0.007828,0.0,0.0,0.0,0.00,0.854902,1.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
54731,0.009785,0.0,0.0,0.0,0.05,0.082353,1.0,0.05,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [308]:
# perform One-hot encoding for the validation set
label_enc = ohe2.transform(df_val.iloc[:,1:4])
label_enc.toarray()
new_labels = ohe2.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_validate = pd.concat([X_validate, df_enc], axis=1)
X_validate

,srv_count,urgent,root_shell,hot,dst_host_srv_diff_host_rate,dst_host_count,logged_in,dst_host_same_src_port_rate,srv_diff_host_rate,num_shells,...,flag_OTH,flag_REJ,flag_RSTO,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.007828,0.0,0.0,0.0,0.01,0.262745,0.0,1.00,0.75,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.039139,0.0,0.0,0.0,0.00,1.000000,1.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.003914,0.0,0.0,0.0,0.00,0.541176,1.0,0.01,1.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.005871,0.0,0.0,0.0,0.02,0.658824,1.0,0.01,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.001957,0.0,0.0,0.0,0.00,0.443137,1.0,0.35,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13652,0.536204,0.0,0.0,0.0,0.00,1.000000,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13653,0.301370,0.0,0.0,0.0,0.00,1.000000,0.0,0.00,0.01,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13654,0.025440,0.0,0.0,0.0,0.00,1.000000,1.0,0.00,0.15,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13655,0.003914,0.0,0.0,0.0,0.00,0.549020,0.0,0.06,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [309]:
# do the same for test set
df_test = df_test[df_test['label'].isin(normal_list+u2r_attacks+r2l_attacks)]

y_test = np.array([0 if x=='normal' else 1 for x in df_test['label']])
df_test = df_test.drop(['label'],axis=1)
df_test = df_test.reset_index().drop(['index'], axis=1)
df_test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,134,86,0.61,0.04,0.61,0.02,0.00,0.00,0.00,0.00
1,0,tcp,http,SF,267,14515,0,0,0,0,...,155,255,1.00,0.00,0.01,0.03,0.01,0.00,0.00,0.00
2,0,tcp,smtp,SF,1022,387,0,0,0,0,...,255,28,0.11,0.72,0.00,0.00,0.00,0.00,0.72,0.04
3,0,tcp,telnet,SF,129,174,0,0,0,0,...,255,255,1.00,0.00,0.00,0.00,0.01,0.01,0.02,0.02
4,0,tcp,http,SF,327,467,0,0,0,0,...,151,255,1.00,0.00,0.01,0.03,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12658,0,tcp,http,SF,274,1623,0,0,0,0,...,92,255,1.00,0.00,0.01,0.04,0.00,0.00,0.00,0.00
12659,0,tcp,http,SF,280,6087,0,0,0,0,...,5,255,1.00,0.00,0.20,0.04,0.00,0.00,0.00,0.00
12660,0,tcp,smtp,SF,794,333,0,0,0,0,...,100,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00
12661,0,tcp,http,SF,317,938,0,0,0,0,...,197,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00


In [310]:
X_test = df_test[common_features_l2] 
X_test

,srv_count,urgent,root_shell,hot,dst_host_srv_diff_host_rate,dst_host_count,logged_in,dst_host_same_src_port_rate,srv_diff_host_rate,num_shells,dst_host_srv_count
0,1,0,0,0,0.02,134,0,0.61,0.00,0,86
1,4,0,0,0,0.03,155,1,0.01,0.00,0,255
2,3,0,0,0,0.00,255,1,0.00,1.00,0,28
3,1,0,0,0,0.00,255,0,0.00,0.00,0,255
4,47,0,0,0,0.03,151,1,0.01,0.04,0,255
...,...,...,...,...,...,...,...,...,...,...,...
12658,1,0,0,0,0.04,92,1,0.01,0.00,0,255
12659,3,0,0,0,0.04,5,1,0.20,0.00,0,255
12660,1,0,0,0,0.01,100,1,0.01,0.00,0,141
12661,11,0,0,0,0.01,197,1,0.01,0.18,0,255


In [311]:
df_minmax = scaler2.transform(X_test)
X_test = pd.DataFrame(df_minmax, columns=X_test.columns)
X_test

,srv_count,urgent,root_shell,hot,dst_host_srv_diff_host_rate,dst_host_count,logged_in,dst_host_same_src_port_rate,srv_diff_host_rate,num_shells,dst_host_srv_count
0,0.001957,0.0,0.0,0.0,0.02,0.525490,0.0,0.61,0.00,0.0,0.337255
1,0.007828,0.0,0.0,0.0,0.03,0.607843,1.0,0.01,0.00,0.0,1.000000
2,0.005871,0.0,0.0,0.0,0.00,1.000000,1.0,0.00,1.00,0.0,0.109804
3,0.001957,0.0,0.0,0.0,0.00,1.000000,0.0,0.00,0.00,0.0,1.000000
4,0.091977,0.0,0.0,0.0,0.03,0.592157,1.0,0.01,0.04,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
12658,0.001957,0.0,0.0,0.0,0.04,0.360784,1.0,0.01,0.00,0.0,1.000000
12659,0.005871,0.0,0.0,0.0,0.04,0.019608,1.0,0.20,0.00,0.0,1.000000
12660,0.001957,0.0,0.0,0.0,0.01,0.392157,1.0,0.01,0.00,0.0,0.552941
12661,0.021526,0.0,0.0,0.0,0.01,0.772549,1.0,0.01,0.18,0.0,1.000000


In [312]:
label_enc = ohe2.transform(df_test.iloc[:,1:4])
label_enc.toarray()
new_labels = ohe2.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_test = pd.concat([X_test, df_enc], axis=1)
X_test

,srv_count,urgent,root_shell,hot,dst_host_srv_diff_host_rate,dst_host_count,logged_in,dst_host_same_src_port_rate,srv_diff_host_rate,num_shells,...,flag_OTH,flag_REJ,flag_RSTO,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.001957,0.0,0.0,0.0,0.02,0.525490,0.0,0.61,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.007828,0.0,0.0,0.0,0.03,0.607843,1.0,0.01,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.005871,0.0,0.0,0.0,0.00,1.000000,1.0,0.00,1.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.001957,0.0,0.0,0.0,0.00,1.000000,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.091977,0.0,0.0,0.0,0.03,0.592157,1.0,0.01,0.04,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12658,0.001957,0.0,0.0,0.0,0.04,0.360784,1.0,0.01,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12659,0.005871,0.0,0.0,0.0,0.04,0.019608,1.0,0.20,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12660,0.001957,0.0,0.0,0.0,0.01,0.392157,1.0,0.01,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12661,0.021526,0.0,0.0,0.0,0.01,0.772549,1.0,0.01,0.18,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [313]:
print('Shape of the train set: ', X_train.shape)
print('Shape of its target: ', y_train.shape)
print('Shape of the test set: ', X_test.shape)
print('Shape of its target: ', y_test.shape)

Shape of the train set:  (54733, 51)
Shape of its target:  (54733,)
Shape of the test set:  (12663, 51)
Shape of its target:  (12663,)


In [314]:
# Under sampling the train set for l2
sm = under_sam(sampling_strategy=1)
X_train, y_train = sm.fit_resample(X_train,y_train)

## Export the datasets
Train set has been scaled, one hot encoded, undersampled
Test set has been scaled and one hot encoded

In [315]:
# Export the dataset for training layer 2
if EXPORT_DATASETS:
    X_train.to_csv('NSL-KDD Encoded Datasets/KDDTrain+_l2.txt', index=False)
    np.save('NSL-KDD Encoded Datasets/KDDTrain+_l2_targets', y_train)

In [316]:
from sklearn import tree

# Principal Component Analysis
pca_r2l_u2r = PCA(n_components=0.95)
X_train_r2l_u2r = pca_r2l_u2r.fit_transform(X_train)
X_test_r2l_u2r = pca_r2l_u2r.transform(X_test)
X_validate_r2l_u2r = pca_r2l_u2r.transform(X_validate)

# Try also Decision Trees
# r2l_u2r_classifier = tree.DecisionTreeClassifier()

# Support Vector Machine for layer l2
r2l_u2r_classifier = SVC(C=0.1, gamma=0.01, kernel='rbf')
r2l_u2r_classifier.fit(X_train_r2l_u2r, y_train)
predicted = r2l_u2r_classifier.predict(X_test_r2l_u2r)

In [317]:
if EXPORT_PCA:
    # save the pca transformed as well as the transformer
    joblib.dump(X_test_r2l_u2r, 'NSL-KDD Encoded Datasets/pca_transformed/pca_test2.pkl')
    joblib.dump(X_train_r2l_u2r, 'NSL-KDD Encoded Datasets/pca_transformed/pca_train2.pkl')
    joblib.dump(X_train_r2l_u2r, 'NSL-KDD Encoded Datasets/pca_transformed/pca_validate2.pkl')
    joblib.dump(pca_r2l_u2r, 'NSL-KDD Encoded Datasets/pca_transformed/layer2_transformer.pkl')

In [318]:
print('Metrics for layer 2:')
print('Confusion matrix: [TP FN / FP TN]\n', confusion_matrix(y_test,predicted))
print('Accuracy = ', accuracy_score(y_test,predicted))
print('F1 Score = ', f1_score(y_test,predicted))
print('Precision = ', precision_score(y_test,predicted))
print('Recall = ', recall_score(y_test,predicted))
print('Matthew corr = ', matthews_corrcoef(y_test,predicted))
print('Shape of the training set: ', X_train_r2l_u2r.shape)

Metrics for layer 2:
Confusion matrix: [TP FN / FP TN]
 [[9036  675]
 [1520 1432]]
Accuracy =  0.8266603490484088
F1 Score =  0.5661197865190749
Precision =  0.6796392975794969
Recall =  0.48509485094850946
Matthew corr =  0.47181218476084225
Shape of the training set:  (1624, 13)


### Export the classifiers

In [319]:
if EXPORT_MODELS:
    with open('Models/NSL_l1_classifier.pkl', "wb") as f:
        pickle.dump(dos_probe_classifier, f)
    with open('Models/NSL_l2_classifier.pkl', "wb") as f:
        pickle.dump(r2l_u2r_classifier, f)

# Testing

In [320]:
df_test1 = copy.deepcopy(df_test_original)
df_test2 = copy.deepcopy(df_test_original)
y_test_real = np.array([0 if x=='normal' else 1 for x in df_test1['label']])
df_test_original

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,private,REJ,0,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
1,0,tcp,private,REJ,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal
22540,0,tcp,http,SF,317,938,0,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal


In [321]:
X_test = df_test1[common_features_l1]
X_test

,logged_in,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_srv_rerror_rate,dst_host_rerror_rate
0,0,229,0.0,1.0,1.0,0.04,0.06,0.00,10,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00
1,0,136,0.0,1.0,1.0,0.01,0.06,0.00,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00
2,0,1,0.0,0.0,0.0,1.00,0.00,0.00,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00
3,0,1,0.0,0.0,0.0,1.00,0.00,1.00,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00
4,0,1,0.0,1.0,0.5,1.00,0.00,0.75,86,0.31,0.17,0.03,0.02,0.00,0.0,0.71,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,1,1,0.0,0.0,0.0,1.00,0.00,0.00,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00
22540,1,2,0.0,0.0,0.0,1.00,0.00,0.18,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00
22541,1,5,0.0,0.0,0.0,1.00,0.00,0.20,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07
22542,0,4,0.0,0.0,0.0,1.00,0.00,0.33,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00


In [322]:
df_minmax = scaler1.transform(X_test)
X_test = pd.DataFrame(df_minmax, columns=X_test.columns)
label_enc = ohe.transform(df_test1.iloc[:,1:4])
label_enc.toarray()
new_labels = ohe.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_test = pd.concat([X_test, df_enc], axis=1)

X_test_layer1 = pca_dos_probe.transform(X_test)
print('Test set shape for layer 1: ', X_test_layer1.shape)

Test set shape for layer 1:  (22544, 28)


In [323]:
X_test = df_test2[common_features_l2] 
X_test

,srv_count,urgent,root_shell,hot,dst_host_srv_diff_host_rate,dst_host_count,logged_in,dst_host_same_src_port_rate,srv_diff_host_rate,num_shells,dst_host_srv_count
0,10,0,0,0,0.00,255,0,0.00,0.00,0,10
1,1,0,0,0,0.00,255,0,0.00,0.00,0,1
2,1,0,0,0,0.02,134,0,0.61,0.00,0,86
3,65,0,0,0,0.28,3,0,1.00,1.00,0,57
4,8,0,0,0,0.02,29,0,0.03,0.75,0,86
...,...,...,...,...,...,...,...,...,...,...,...
22539,1,0,0,0,0.01,100,1,0.01,0.00,0,141
22540,11,0,0,0,0.01,197,1,0.01,0.18,0,255
22541,10,0,0,2,0.00,255,1,0.00,0.20,0,255
22542,6,0,0,0,0.00,255,0,0.00,0.33,0,252


In [324]:
df_minmax = scaler2.transform(X_test)
X_test = pd.DataFrame(df_minmax, columns=X_test.columns)
label_enc = ohe2.transform(df_test2.iloc[:,1:4])
label_enc.toarray()
new_labels = ohe2.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_test = pd.concat([X_test, df_enc], axis=1)

X_test_layer2 = pca_r2l_u2r.transform(X_test)
print('Test set shape for layer 2: ', X_test_layer2.shape)
print('Type of X_test_layer1: ', type(X_test_layer1))
print('Type of X_test_layer1: ', type(X_test_layer2))

Test set shape for layer 2:  (22544, 13)
Type of X_test_layer1:  <class 'numpy.ndarray'>
Type of X_test_layer1:  <class 'numpy.ndarray'>


In [325]:
# same classifiers obtained above
classifier1 = dos_probe_classifier
classifier2 = r2l_u2r_classifier

In [326]:
result = []
for i in range(X_test_layer2.shape[0]):
    layer1 = classifier1.predict(X_test_layer1[i].reshape(1, -1))[0]
    if layer1 == 1:
        result.append(layer1)
    else:
        layer2 = classifier2.predict(X_test_layer2[i].reshape(1, -1))[0]
        if layer2 == 1:
            result.append(layer2)
        else:
            result.append(0)
            
result = np.array(result)

In [327]:
# the results may vary
# C=0.1, gamma=0.01
print('Results for the layer 2 (SVM):')
print(confusion_matrix(y_test_real,result))
print('Accuracy = ', accuracy_score(y_test_real,result))
print('F1 Score = ', f1_score(y_test_real,result))
print('Precision = ', precision_score(y_test_real,result))
print('Recall = ', recall_score(y_test_real,result))
print('Matthew corr = ', matthews_corrcoef(y_test_real,result))

Results for the layer 2 (SVM):
[[ 8107  1604]
 [  899 11934]]
Accuracy =  0.888972675656494
F1 Score =  0.9050851313943346
Precision =  0.8815186881370956
Recall =  0.9299462323696719
Matthew corr =  0.7731882307380973


### Export the test sets

In [328]:
if EXPORT_DATASETS:
    column_names = [f'PC{i}' for i in range(1, X_test_layer1.shape[1] + 1)]
    X1_test = pd.DataFrame(data=X_test_layer1, columns=column_names)
    X1_test.to_csv('NSL-KDD Encoded Datasets/X_test_l1.txt', index=False)
    
    column_names = [f'PC{i}' for i in range(1, X_test_layer2.shape[1] + 1)]
    X2_test = pd.DataFrame(data=X_test_layer2, columns=column_names)
    X2_test.to_csv('NSL-KDD Encoded Datasets/X_test_l2.txt', index=False)
    
    np.save('NSL-KDD Encoded Datasets/y_test', y_test_real)

### evaluate seen and unseen attack categories

In [329]:
# load testset
df_test = pd.read_csv('NSL-KDD Original Datasets\KDDTest+.txt', sep=",", header=None)
df_test = df_test[df_test.columns[:-1]]
df_test.columns = titles.to_list()
y_test = df_test['label']
df_test = df_test.drop(['num_outbound_cmds'],axis=1)
df_test_original = df_test

In [330]:
if EXPORT_DATASETS:
    df_test_original.to_csv('NSL-KDD Encoded Datasets/KDDTest+', index=False)
    np.save('NSL-KDD Encoded Datasets/KDDTest+_targets', y_test)
    
df_test_original

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,private,REJ,0,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
1,0,tcp,private,REJ,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal
22540,0,tcp,http,SF,317,938,0,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal


In [331]:
new_attack = []
for i in df_test_original['label'].value_counts().index.tolist()[1:]:
    if i not in df_train_original['label'].value_counts().index.tolist()[1:]:
        new_attack.append(i)
        
new_attack.sort()
new_attack

['apache2',
 'httptunnel',
 'mailbomb',
 'mscan',
 'named',
 'processtable',
 'ps',
 'saint',
 'sendmail',
 'snmpgetattack',
 'snmpguess',
 'sqlattack',
 'udpstorm',
 'worm',
 'xlock',
 'xsnoop',
 'xterm']

In [332]:
index_of_new_attacks = []

for i in range(len(df_test_original)):
    if df_test_original['label'][i] in new_attack:
        index_of_new_attacks.append(df_test_original.index[i])

In [333]:
len(index_of_new_attacks)

3750

In [334]:
new_attack.append('normal')
new_attack

['apache2',
 'httptunnel',
 'mailbomb',
 'mscan',
 'named',
 'processtable',
 'ps',
 'saint',
 'sendmail',
 'snmpgetattack',
 'snmpguess',
 'sqlattack',
 'udpstorm',
 'worm',
 'xlock',
 'xsnoop',
 'xterm',
 'normal']

In [335]:
index_of_old_attacks = []

for i in range(len(df_test_original)):
    if df_test_original['label'][i] not in new_attack:
        index_of_old_attacks.append(df_test_original.index[i])

In [336]:
len(index_of_old_attacks)

9083

In [337]:
print('Number of new attacks in the test set: ', result[index_of_new_attacks].shape[0])
print('Number of new attacks detected by the classifiers: ', result[index_of_new_attacks].sum())
print('Proportion of new attacks detected: ', result[index_of_new_attacks].sum()/result[index_of_new_attacks].shape[0])

Number of new attacks in the test set:  3750
Number of new attacks detected by the classifiers:  3416
Proportion of new attacks detected:  0.9109333333333334


In [338]:
print('Number of old attacks in the test set: ', result[index_of_old_attacks].shape[0])
print('Number of old attacks detected by the classifiers: ', result[index_of_old_attacks].sum())
print('Proportion of old attacks detected: ', result[index_of_old_attacks].sum()/result[index_of_old_attacks].shape[0])

Number of old attacks in the test set:  9083
Number of old attacks detected by the classifiers:  8518
Proportion of old attacks detected:  0.9377958824177034


### Evaluate single attack types

In [339]:
# load test set
df_test = pd.read_csv('NSL-KDD Original Datasets/KDDTest+.txt', sep=",", header=None)
df_test = df_test[df_test.columns[:-1]]
df_test.columns = titles.to_list()
y_test = df_test['label']
df_test = df_test.drop(['num_outbound_cmds'],axis=1)
df_test_original = df_test
df = df_test_original

dos_index = df.index[(df['label'].isin(dos_attacks))].tolist()
probe_index = df.index[(df['label'].isin(probe_attacks))].tolist()
r2l_index = df.index[(df['label'].isin(r2l_attacks))].tolist()
u2r_index = df.index[(df['label'].isin(u2r_attacks))].tolist()

print('Evaluation split into single attack type:')
print("Number of dos attacks: ", result[dos_index].shape[0])
print("Number of detected attacks: ", result[dos_index].sum())
print("Ratio of detection: ", result[dos_index].sum()/result[dos_index].shape[0])

print("Number of probe attacks: ", result[probe_index].shape[0])
print("Number of detected attacks: ", result[probe_index].sum())
print("Ratio of detection: ", result[probe_index].sum()/result[probe_index].shape[0])

print("Number of r2l attacks: ", result[r2l_index].shape[0])
print("Number of detected attacks: ", result[r2l_index].sum())
print("Ratio of detection: ", result[r2l_index].sum()/result[r2l_index].shape[0])

print("Number of u2r attacks: ", result[u2r_index].shape[0])
print("Number of detected attacks: ", result[u2r_index].sum())
print("Ratio of detection: ", result[u2r_index].sum()/result[u2r_index].shape[0])

Evaluation split into single attack type:
Number of dos attacks:  7460
Number of detected attacks:  6880
Ratio of detection:  0.9222520107238605
Number of probe attacks:  2421
Number of detected attacks:  2212
Ratio of detection:  0.9136720363486163
Number of r2l attacks:  2885
Number of detected attacks:  2775
Ratio of detection:  0.9618717504332756
Number of u2r attacks:  67
Number of detected attacks:  67
Ratio of detection:  1.0


In [340]:
# Export one hot encoders
if EXPORT_ENCODERS:
    joblib.dump(ohe, 'NSL-KDD Files/one_hot_encoders/ohe1.pkl')
    joblib.dump(ohe2, 'NSL-KDD Files/one_hot_encoders/ohe2.pkl')